# Desafio 5

Neste desafio, vamos praticar sobre redução de dimensionalidade com PCA e seleção de variáveis com RFE. Utilizaremos o _data set_ [Fifa 2019](https://www.kaggle.com/karangadiya/fifa19), contendo originalmente 89 variáveis de mais de 18 mil jogadores do _game_ FIFA 2019.

> Obs.: Por favor, não modifique o nome das funções de resposta.

## _Setup_ geral

In [1]:
from math import sqrt

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as sct
import seaborn as sns
import statsmodels.api as sm
import statsmodels.stats as st
from sklearn.decomposition import PCA

from loguru import logger

C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# Algumas configurações para o matplotlib.
#%matplotlib inline

from IPython.core.pylabtools import figsize


figsize(12, 8)

sns.set()

In [3]:
fifa = pd.read_csv("fifa.csv")

In [4]:
columns_to_drop = ["Unnamed: 0", "ID", "Name", "Photo", "Nationality", "Flag",
                   "Club", "Club Logo", "Value", "Wage", "Special", "Preferred Foot",
                   "International Reputation", "Weak Foot", "Skill Moves", "Work Rate",
                   "Body Type", "Real Face", "Position", "Jersey Number", "Joined",
                   "Loaned From", "Contract Valid Until", "Height", "Weight", "LS",
                   "ST", "RS", "LW", "LF", "CF", "RF", "RW", "LAM", "CAM", "RAM", "LM",
                   "LCM", "CM", "RCM", "RM", "LWB", "LDM", "CDM", "RDM", "RWB", "LB", "LCB",
                   "CB", "RCB", "RB", "Release Clause"
]

try:
    fifa.drop(columns_to_drop, axis=1, inplace=True)
except KeyError:
    logger.warning(f"Columns already dropped")

## Inicia sua análise a partir daqui

In [5]:
# Sua análise começa aqui.

# Inspecionando o dataset
fifa.columns

Index(['Age', 'Overall', 'Potential', 'Crossing', 'Finishing',
       'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling', 'Curve',
       'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle', 'GKDiving', 'GKHandling',
       'GKKicking', 'GKPositioning', 'GKReflexes'],
      dtype='object')

In [6]:
print(fifa.dtypes)

Age                  int64
Overall              int64
Potential            int64
Crossing           float64
Finishing          float64
HeadingAccuracy    float64
ShortPassing       float64
Volleys            float64
Dribbling          float64
Curve              float64
FKAccuracy         float64
LongPassing        float64
BallControl        float64
Acceleration       float64
SprintSpeed        float64
Agility            float64
Reactions          float64
Balance            float64
ShotPower          float64
Jumping            float64
Stamina            float64
Strength           float64
LongShots          float64
Aggression         float64
Interceptions      float64
Positioning        float64
Vision             float64
Penalties          float64
Composure          float64
Marking            float64
StandingTackle     float64
SlidingTackle      float64
GKDiving           float64
GKHandling         float64
GKKicking          float64
GKPositioning      float64
GKReflexes         float64
d

Todos dados numéricos, excelente dataset :)

In [7]:
# analisando colunas nulas, PCA não mastiga bem estas colunas

fifa.isnull().sum()

Age                 0
Overall             0
Potential           0
Crossing           48
Finishing          48
HeadingAccuracy    48
ShortPassing       48
Volleys            48
Dribbling          48
Curve              48
FKAccuracy         48
LongPassing        48
BallControl        48
Acceleration       48
SprintSpeed        48
Agility            48
Reactions          48
Balance            48
ShotPower          48
Jumping            48
Stamina            48
Strength           48
LongShots          48
Aggression         48
Interceptions      48
Positioning        48
Vision             48
Penalties          48
Composure          48
Marking            48
StandingTackle     48
SlidingTackle      48
GKDiving           48
GKHandling         48
GKKicking          48
GKPositioning      48
GKReflexes         48
dtype: int64

In [8]:
# calculando o percentual de dados nulos por coluna

fifa.isnull().sum()/fifa.shape[0]

Age                0.000000
Overall            0.000000
Potential          0.000000
Crossing           0.002636
Finishing          0.002636
HeadingAccuracy    0.002636
ShortPassing       0.002636
Volleys            0.002636
Dribbling          0.002636
Curve              0.002636
FKAccuracy         0.002636
LongPassing        0.002636
BallControl        0.002636
Acceleration       0.002636
SprintSpeed        0.002636
Agility            0.002636
Reactions          0.002636
Balance            0.002636
ShotPower          0.002636
Jumping            0.002636
Stamina            0.002636
Strength           0.002636
LongShots          0.002636
Aggression         0.002636
Interceptions      0.002636
Positioning        0.002636
Vision             0.002636
Penalties          0.002636
Composure          0.002636
Marking            0.002636
StandingTackle     0.002636
SlidingTackle      0.002636
GKDiving           0.002636
GKHandling         0.002636
GKKicking          0.002636
GKPositioning      0

0.2% de dados faltantes, ire dropar estas linhas

In [9]:
# bye
fifa.dropna(inplace=True)


## Questão 1

Qual fração da variância consegue ser explicada pelo primeiro componente principal de `fifa`? Responda como um único float (entre 0 e 1) arredondado para três casas decimais.

Receita de bolo de como usar o [scikit-learn](https://jakevdp.github.io/PythonDataScienceHandbook/05.02-introducing-scikit-learn.html)

In [10]:
# reduzindo para dois componentes principais

pca_2comp = PCA(n_components=2)

# fit_transform para centralizar os dados de entrada
pca_2comp.fit_transform(fifa)



array([[-126.71792515, -105.58000764],
       [-123.36568604,  -88.98416141],
       [-115.11013638,  -94.77505594],
       ...,
       [  49.83476757,  -44.19006161],
       [  42.8077879 ,  -39.12437286],
       [  37.17937197,   15.60717604]])

Vale a pena olhar a documentação para entender os atributos do [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html?highlight=pca#sklearn.decomposition.PCA)

In [11]:
print(pca_2comp.components_)

print(pca_2comp.explained_variance_)

print(pca_2comp.explained_variance_ratio_)

[[-6.16388751e-03 -3.70623864e-02 -2.27401748e-02 -2.13639023e-01
  -1.98891213e-01 -1.70828950e-01 -1.80309140e-01 -1.87038764e-01
  -2.33139606e-01 -2.07690956e-01 -1.84764187e-01 -1.67186902e-01
  -2.12972623e-01 -1.38740617e-01 -1.34902279e-01 -1.38433521e-01
  -4.91611013e-02 -1.16410947e-01 -1.95840156e-01 -4.07181861e-02
  -1.72711671e-01 -2.33585866e-02 -2.16594765e-01 -1.53339724e-01
  -1.50834334e-01 -2.24571087e-01 -1.29586783e-01 -1.62548283e-01
  -1.01038031e-01 -1.53454113e-01 -1.59617493e-01 -1.47955869e-01
   2.06147192e-01  1.96645602e-01  1.91129889e-01  1.97401130e-01
   2.08994083e-01]
 [ 8.87203494e-03  1.58367355e-04 -7.78142440e-03 -4.43084573e-02
  -2.57629630e-01  1.18911964e-01  1.21869793e-02 -1.91182282e-01
  -1.18898465e-01 -1.27744634e-01 -1.00178915e-01  4.89136910e-02
  -5.12678591e-02 -9.84027581e-02 -8.39018967e-02 -1.21564730e-01
   9.67807471e-04 -9.04417070e-02 -1.04656794e-01  5.73651299e-02
   6.82049478e-02  1.19388946e-01 -1.73409231e-01  2.1334

In [12]:
def q1():
    # Retorne aqui o resultado da questão 1.
    return float(round(pca_2comp.explained_variance_ratio_[0],3))

## Questão 2

Quantos componentes principais precisamos para explicar 95% da variância total? Responda como un único escalar inteiro.

In [13]:
# mantendo 95% da variancia
pca_95 = PCA(0.95)
pca_95.fit_transform(fifa)
print(pca_95.n_components_)

#confirma ?
print(np.cumsum(pca_95.explained_variance_ratio_))

15
[0.56528056 0.74630577 0.80583411 0.84926104 0.87561427 0.88768504
 0.89804009 0.9065696  0.91444823 0.9216524  0.9288342  0.93516314
 0.94035556 0.94526355 0.95000564]


In [14]:
def q2():
    # Retorne aqui o resultado da questão 2.
    return int(pca_95.n_components_)

## Questão 3

Qual são as coordenadas (primeiro e segundo componentes principais) do ponto `x` abaixo? O vetor abaixo já está centralizado. Cuidado para __não__ centralizar o vetor novamente (por exemplo, invocando `PCA.transform()` nele). Responda como uma tupla de float arredondados para três casas decimais.

In [15]:
x = [0.87747123,  -1.24990363,  -1.3191255, -36.7341814,
     -35.55091139, -37.29814417, -28.68671182, -30.90902583,
     -42.37100061, -32.17082438, -28.86315326, -22.71193348,
     -38.36945867, -20.61407566, -22.72696734, -25.50360703,
     2.16339005, -27.96657305, -33.46004736,  -5.08943224,
     -30.21994603,   3.68803348, -36.10997302, -30.86899058,
     -22.69827634, -37.95847789, -22.40090313, -30.54859849,
     -26.64827358, -19.28162344, -34.69783578, -34.6614351,
     48.38377664,  47.60840355,  45.76793876,  44.61110193,
     49.28911284
]

# Basta decompor o vertor na direção das duas componentes principais calculadas anteriormente
# z = Ureduce' * x  
# para um componente é o produto interno dos vetores
result = np.dot(pca_2comp.components_,x)

# Aredondando e criando a tupla para resposta
tuple([round(x,3) if isinstance(x, float) else x for x in result])



(186.556, -6.592)

In [16]:
def q3():
    # Retorne aqui o resultado da questão 3.
    return tuple([round(x,3) if isinstance(x, float) else x for x in result])

## Questão 4

Realiza RFE com estimador de regressão linear para selecionar cinco variáveis, eliminando uma a uma. Quais são as variáveis selecionadas? Responda como uma lista de nomes de variáveis.

Motivação para escolha do target ser 'Overall' está descrita neste [entendendo os atributos](http://comufifa.blogspot.com/2012/05/fifa-entendendo-os-atributos.html) do FIFA!

Valeu pela dica galera da [comunidade Codenation](https://comunidade.codenation.dev/login) de Data Science - Abr/2020


In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE

# preparando a matriz de features
X_feat = fifa.drop(columns='Overall')

# estimador de regressão linear com cinco variáveis
model = LinearRegression()
rfe = RFE(model, 5)

# Target é a coluna 'Overall'
rfe = rfe.fit(X_feat, fifa['Overall'])

print(rfe.support_)

print(rfe.n_features_)

print(rfe.ranking_)

[ True  True False False False False False False False False False  True
 False False False  True False False False False False False False False
 False False False False False False False False False False False  True]
5
[ 1  1 17 19  7  5 32 23 24 26 27  1 12  6 28  1 13 21 30  9  2 25 29 31
 11 18 22  3 20 15 16  4  8 10 14  1]


Usando o atributo support_ como máscara... e convertendo para lista como requerido.

In [18]:
lista = X_feat.columns[rfe.support_].to_list()
lista

['Age', 'Potential', 'BallControl', 'Reactions', 'GKReflexes']

In [19]:
def q4():
    # Retorne aqui o resultado da questão 4.
    return X_feat.columns[rfe.support_].to_list()

Esse desafio foi uma saga! 
mas divertido.